In [10]:
%run package_import.ipynb
%run data_profiling.ipynb

In [2]:
from sqlalchemy import create_engine
import pymysql

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/kaggle_hour_price', pool_recycle=3600)
dbConnection    = sqlEngine.connect()

In [21]:
import os
os.chdir(path='/Users/liuyifu/Project_data')

In [120]:
df_combined = pd.read_sql_query(

''' 
select *,
    'train' as source
from raw_train

union all

select *, 
    cast(null as float) as SalePrice,
    'test' as source
from raw_test

'''
,dbConnection)

In [125]:
main_data_profiling(df_combined)

,column_names,data_type,unique_count,min_value,max_value,mean_value,median_value,null_values,total_rows,%_null_values,num_lower_outliers,num_higher_outliers
0,Id,int64,2919,1.0,2919.0,1460.000000,1460.0,0,2919,0.000000,0.0,0.0
1,MSSubClass,int64,16,20.0,190.0,57.137718,50.0,0,2919,0.000000,0.0,207.0
2,MSZoning,object,6,NaN,NaN,NaN,NaN,4,2919,0.001370,NaN,NaN
3,LotFrontage,float64,129,21.0,313.0,69.305795,68.0,486,2919,0.166495,104.0,94.0
4,LotArea,int64,1951,1300.0,215245.0,10168.114080,9453.0,0,2919,0.000000,1.0,127.0
5,Street,object,2,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
6,Alley,object,3,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
7,LotShape,object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
8,LandContour,object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
9,Utilities,object,3,NaN,NaN,NaN,NaN,2,2919,0.000685,NaN,NaN


### ETL Process

In [131]:
# The loop is to convert description to jason to fill na value
# initalise the dictionary
desc_dict, attr_dict, col_desc = {}, {}, {}
with open('House_Prices/data_description.txt', encoding='utf8') as f:
    for line in f:
        # convert line into list
        words = line.split()
        # if the line contains ":", the line includes a column
        # and its description
        if words and ':' in line:
            # convert the line into list, recognise the first item from the list
            # as the column and remove ":"
            col = words[0].replace(':','')
            # save the column as a key into description dictionary
            desc_dict[col] = {}
            col_desc[col] = ' '.join(words[1:])
        # if the line is blank, move to the next one
        elif  not words:
            continue
        else:
            # the attr_desc is to contain values for each column
            # concatenate items except the value from the list to 
            # rebuild the description of each value
            attr_desc = ' '.join(words[1:])
            # store the first item of the list as the value
            attr = words[0]
            # save the the attribute dictionary as sub-dictionary to
            # description description
            desc_dict[col][attr] = attr_desc

In [136]:
df_profile = main_data_profiling(df_combined)
df_profile.insert(1, 'col_desc', np.nan)
df_profile['col_desc'] = df_profile['column_names'].map(col_desc)
df_profile

,column_names,col_desc,data_type,unique_count,min_value,max_value,mean_value,median_value,null_values,total_rows,%_null_values,num_lower_outliers,num_higher_outliers
0,Id,NaN,int64,2919,1.0,2919.0,1460.000000,1460.0,0,2919,0.000000,0.0,0.0
1,MSSubClass,Identifies the type of dwelling involved in the sale.,int64,16,20.0,190.0,57.137718,50.0,0,2919,0.000000,0.0,207.0
2,MSZoning,Identifies the general zoning classification of the sale.,object,6,NaN,NaN,NaN,NaN,4,2919,0.001370,NaN,NaN
3,LotFrontage,Linear feet of street connected to property,float64,129,21.0,313.0,69.305795,68.0,486,2919,0.166495,104.0,94.0
4,LotArea,Lot size in square feet,int64,1951,1300.0,215245.0,10168.114080,9453.0,0,2919,0.000000,1.0,127.0
5,Street,Type of road access to property,object,2,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
6,Alley,Type of alley access to property,object,3,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
7,LotShape,General shape of property,object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
8,LandContour,Flatness of the property,object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
9,Utilities,Type of utilities available,object,3,NaN,NaN,NaN,NaN,2,2919,0.000685,NaN,NaN


#### 1. Replace NA

In [124]:
# the loop is to fill na value based on descriptioin automatically
for col in df_combined.columns:
    # if a column has null value and its value includes "NA"
    # replace NA with the value mentioned in the description;
    # otherwise, skip it. 
    if df_profile[df_profile['column_names'] == col]['null_values'].squeeze() > 0:
        if col in desc_dict and desc_dict[col].get('NA'):
            df_combined[col] = df_combined[col].fillna(desc_dict[col]['NA'])

#### 2. Check duplication

In [127]:
df_combined[df_combined.duplicated()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,source


### Upload transformed data

In [141]:
df_combined.to_sql('transformed_combined', dbConnection, if_exists='replace', index=False)